In [1]:
import os

# Set env vars BEFORE importing huggingface modules
os.environ["HF_HOME"] = "/projects/sciences/computing/sheju347/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/projects/sciences/computing/sheju347/.cache/huggingface/hub"

# Now import huggingface modules
from huggingface_hub import constants

print("HF_HOME:", constants.HF_HOME)
print("HF_HUB_CACHE:", constants.HF_HUB_CACHE)

HF_HOME: /projects/sciences/computing/sheju347/.cache/huggingface
HF_HUB_CACHE: /projects/sciences/computing/sheju347/.cache/huggingface/hub


In [2]:

class TrainingData:
    def __init__(self, context: str, question: str, is_correct: bool):
        self.context = context
        self.question = question
        self.is_correct = is_correct


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# model_name = "google/flan-t5-large"  # or t5-base, t5-large
# model_name = "google-t5/t5-base"
# model_name = "google-t5/t5-small"
model_name = "google-t5/t5-large"
# model_name = "/projects/sciences/computing/sheju347/RAG/classifier/t5-large-epoch-10/checkpoint-94290"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
print("done")

/projects/sciences/computing/sheju347/miniconda3/envs/LLM311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/projects/sciences/computing/sheju347/miniconda3/envs/LLM311/lib/python3.11/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


done


In [7]:
import re
import os

QUESTION_COUNT = 10178

# log_file_list = [
#     "9-20-train_150_30_1st_H100.txt", 
#     "9-20-train_150_30_2nd_H100.txt", 
#     "9-21-train_150_30_3rd_H100.txt",
#     "9-23-train_150_30_4th_H100.txt",
#     "9-24-train_150_30_5th_H100.txt",
#                 ]
# log_file = "9-20-train_150_30_2nd_H100.txt"
training_file_list = ["10-14-lora-context012-r16-epoch3-150-30-5-classifier-train.txt"]
test_file_list = ["10-14-lora-context012-r16-epoch3-150-30-5-classifier-test.txt"]

# path = "../../notebooks/"
path = "./"

training_file_list = [path + log_file for log_file in training_file_list]
test_file_list = [path + log_file for log_file in test_file_list]

training_data_list = []
test_data_list = []

def get_data_list(file_name_list):
    ret_data_list = []
    for file_name in file_name_list:
        with open(file_name) as f:
            is_reading_context = False
            context = ""
        
            is_reading_question = False
            question = ""
        
            for line in f:
                # if line == "\n":
                #     continue
        
                stripped_line = line.strip()
                if stripped_line == "Context:":
                    is_reading_context = True
                elif stripped_line == "Question:":
                    is_reading_context = False
                    is_reading_question = True
                elif stripped_line.startswith("[output]"):
                    is_correct = stripped_line[len("[output]"):] == "True"
                    context = context.strip()
                    question = question.strip()
                    data = TrainingData(context, question, is_correct)
                    # print("aaa", context, question, is_correct)
        
                    ret_data_list.append(data)
        
                    # if len(training_data_list) % 100 == 0:
                    #     print(f"finished processing {len(training_data_list)} data")
        
                    # clear
                    is_reading_context = False
                    context = ""
                    is_reading_question = False
                    question = ""
                    # break
                else:
                    if is_reading_context:
                        context += line
                    elif is_reading_question:
                        question += line
                        
    return ret_data_list

training_data_list = get_data_list(training_file_list)
test_data_list = get_data_list(test_file_list)

# training_data_list = training_data_list[:1000]
print("---training---")
print(len(training_data_list))
print(training_data_list[1].context, training_data_list[1].is_correct)
print("---test---")
print(len(test_data_list))
print(test_data_list[1].context, test_data_list[1].is_correct)

---training---
6754
Gastric cancer presenting as a krukenberg tumor at 22 weeks' gestation. Gastric cancer is rare during pregnancy, and often advanced upon presentation. A Krukenberg tumor presents a diagnostic and therapeutic challenge in the pregnant patient. We present a case of a 38-year-old woman at 22 weeks' gestation who presented with worsening epigastric pain, and was found to have a left pelvic mass on ultrasound, which was confirmed by magnetic resonance imaging. She went into active labor and delivered a viable infant via vaginal delivery. An exploratory laparotomy revealed a large mass originating from her left ovary and diffuse thickening of the lesser curvature of the stomach. Frozen section investigation revealed the presence of signet cell adenocarcinoma. Subsequent upper endoscopy showed linitis plastica, while biopsy confirmed the presence of adenocarcinoma. In conclusion, the occurrence of gastric cancer in pregnancy is rare despite extremely common symptoms. The m

In [8]:
# Force training data to be balanced
adapted_data_list = []
incorrect_count = 0
for data in training_data_list:
    if data.is_correct == False:
        incorrect_count += 1

counter = 0
for data in training_data_list:
    if data.is_correct == False:
        adapted_data_list.append(data)
    else:
        if counter < incorrect_count + 0:
            adapted_data_list.append(data)
            counter += 1

training_data_list = adapted_data_list


# Change the label based on length
lengths = []
for data in training_data_list:
    lengths.append(len(data.context) + len(data.question))
import statistics
median_len = statistics.median(lengths)
print("median_len", median_len)

# for data in training_data_list:
#     if (len(data.context) + len(data.question)) > median_len:
#         data.is_correct = True
#     else:
#         data.is_correct = False
    

false_count = 0
true_count = 0
for data in training_data_list:
    if data.is_correct:
        true_count += 1
    else:
        false_count += 1
print(false_count, true_count)

median_len 2332.0
1493 1493


In [ ]:
filename = "./data/cfimdb-train.txt"

training_data_list = []

with open(filename, 'r') as fp:
    for line in fp:
        label, org_sent = line.split(' ||| ')
        # sent = org_sent.lower().strip()
        # tokens = tokenizer.tokenize("[CLS] " + sent + " [SEP]")
        label = int(label.strip())
        # if label not in num_labels:
        #     num_labels[label] = len(num_labels)
        data = TrainingData(org_sent, "", label)
        training_data_list.append(data)

print(training_data_list[6].context, training_data_list[6].is_correct)

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Suppose training_data_list is your list of TrainingData objects
training_dataframe = pd.DataFrame([{
    "context": d.context,
    "question": d.question,
    "label": int(d.is_correct)   # True->1, False->0
} for d in training_data_list])

test_dataframe = pd.DataFrame([{
    "context": d.context,
    "question": d.question,
    "label": int(d.is_correct)   # True->1, False->0
} for d in test_data_list])

# # Train/val/test split (e.g., 70/15/15)
# train_df, temp_df = train_test_split(data, test_size=0.3, random_state=42, stratify=data["label"])
# val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"])

# print(len(train_df), len(val_df), len(test_df))
# # print(test_df.iloc[0]["context"])
# # print(test_df.iloc[0]["question"])
# print(test_df.iloc[1]["label"])

In [11]:
import torch
from torch.utils.data import Dataset

MAX_LEN = 1024 #512
TARGET_MAX_LEN = 2

class QADataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=MAX_LEN, target_max_len=TARGET_MAX_LEN):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.target_max_len = target_max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context = self.data.loc[idx, "context"]
        question = self.data.loc[idx, "question"]
        label = self.data.iloc[idx]["label"]

        input_text = f"question: {question} context: {context}"
        target_text = "yes" if label == 1 else "no"

        # Keep return_tensors=None, let __getitem__ return 1D tensors
        encoding = self.tokenizer(
            input_text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors=None
        )
        
        target_encoding = self.tokenizer(
            target_text,
            truncation=True,
            padding="max_length",
            max_length=self.target_max_len,
            return_tensors=None
        )

        # Convert to tensors (1D)
        input_ids = torch.tensor(encoding["input_ids"], dtype=torch.long)
        attention_mask = torch.tensor(encoding["attention_mask"], dtype=torch.long)
        labels = torch.tensor(target_encoding["input_ids"], dtype=torch.long)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

In [12]:
# train_dataset = QADataset(train_df, tokenizer)
# val_dataset = QADataset(val_df, tokenizer)
# test_dataset = QADataset(test_df, tokenizer)

train_dataset = QADataset(training_dataframe, tokenizer)
test_dataset = QADataset(test_dataframe, tokenizer)

In [15]:
print(train_dataset[9]["input_ids"].tolist())
# print(len(train_dataset[2]["input_ids"].tolist()))
# print(train_dataset[1]["labels"])

[822, 10, 71, 5354, 7393, 18, 1490, 1871, 3977, 8247, 44, 706, 298, 17915, 5, 978, 2039, 4944, 24, 3, 88, 141, 3977, 163, 227, 255, 3, 9, 18423, 16, 8, 1379, 5, 465, 1137, 13, 1687, 47, 4187, 3, 390, 30, 8, 1510, 19819, 5, 4073, 13, 8, 826, 21059, 7, 228, 43, 19653, 8, 1687, 13, 8, 1871, 58, 784, 188, 908, 3, 10, 8422, 75, 53, 8, 9806, 16, 3, 9, 3, 7, 413, 630, 1102, 30, 3, 9, 1669, 8841, 298, 9182, 784, 279, 908, 3, 10, 3, 18536, 8, 9806, 2303, 11, 6011, 3, 9, 306, 562, 2912, 784, 254, 908, 3, 10, 8566, 13, 3, 9, 1407, 12, 1961, 8, 9182, 1102, 784, 308, 908, 3, 10, 15856, 53, 3, 5379, 7903, 169, 383, 2085, 2625, 10, 71, 495, 13, 12914, 9806, 1687, 788, 12, 3, 9, 2329, 16643, 3, 7, 291, 287, 9, 5, 37, 495, 2196, 270, 77, 3315, 8, 7544, 1687, 13, 3, 9, 5354, 7393, 18, 1490, 3955, 21118, 113, 8247, 11612, 26, 16, 160, 2039, 31, 7, 6026, 11, 47, 3654, 30, 6870, 44, 8, 2833, 5, 37, 3739, 27771, 13, 8, 1871, 11, 160, 1362, 130, 2841, 21, 3976, 42, 3957, 13, 7095, 6, 237, 273, 13, 16712, 523

In [16]:
from transformers import Trainer, TrainingArguments


output_dir = "/projects/sciences/computing/sheju347/RAG/classifier/10-19-t5-large-epoch-10" # "./results"

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# def compute_metrics(eval_pred):
#     from sklearn.metrics import accuracy_score, f1_score
#     logits, labels = eval_pred
#     preds = logits.argmax(-1)
#     return {
#         "accuracy": accuracy_score(labels, preds),
#         "f1": f1_score(labels, preds)
#     }

def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, f1_score
    import torch

    # Extract logits and labels
    logits = eval_pred.predictions
    labels = eval_pred.label_ids

    # If logits is a tuple (common for seq2seq models), take the first element
    if isinstance(logits, tuple):
        logits = logits[0]

    # Convert logits to predicted token IDs
    pred_ids = torch.argmax(torch.tensor(logits), dim=-1)
    
    # Decode to text
    decoded_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    
    # Map yes/no to 1/0
    y_pred = [1 if x.strip().lower() == "yes" else 0 for x in decoded_preds]
    
    # True labels
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    y_true = [1 if x.strip().lower() == "yes" else 0 for x in decoded_labels]

    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred)
    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipykernel_2593613/1349905499.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.367200,0.970250,0.437363,0.493069
2,0.244700,2.725734,0.352747,0.320646
3,0.239700,1.591802,0.409890,0.461384
4,0.165000,1.716623,0.380220,0.423313
5,0.138800,2.293863,0.343956,0.298472
6,0.152400,2.081471,0.363736,0.364435
7,0.081300,1.632737,0.457143,0.548446
8,0.081500,1.836248,0.424176,0.488281
9,0.067400,2.111387,0.383516,0.403826
10,0.082700,2.384259,0.400000,0.450704


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

# model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"
# model_name = "google-bert/bert-base-uncased"
# model_name = "allenai/longformer-base-4096"
model_name = "/projects/sciences/computing/sheju347/RAG/classifier/10-19-t5-large-epoch-10/checkpoint-29860"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
from sklearn.metrics import accuracy_score

preds = trainer.predict(test_dataset)
print(preds.metrics)  # optional, shows overall metrics

# Get logits
logits = preds.predictions  # <--- use .predictions


# If logits is a tuple (common for seq2seq models), take the first element
if isinstance(logits, tuple):
    logits = logits[0]  # shape: (num_examples, seq_len, vocab_size)

print("logits", logits)
# Convert logits to token IDs
pred_ids = torch.argmax(torch.tensor(logits), dim=-1)

print ("pred_ids", pred_ids)
# Decode token IDs to text
decoded_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(preds.label_ids, skip_special_tokens=True)

print("decoded_preds", decoded_preds)
print("decoded_labels", decoded_labels)

# Convert yes/no to 1/0
y_pred = [1 if x.strip().lower() == "yes" else 0 for x in decoded_preds]
y_true = [1 if x.strip().lower() == "yes" else 0 for x in decoded_labels]

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)


In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
import torch
from torch.utils.data import Subset

# trainer.args.predict_with_generate = False  # make sure no .generate()

chunk_size = 500  # adjust depending on your GPU memory
all_logits = []
all_labels = []

for start in range(0, len(test_dataset), chunk_size):
    end = min(start + chunk_size, len(test_dataset))
    subset = Subset(test_dataset, range(start, end))
    preds = trainer.predict(subset)

    logits = preds.predictions
    if isinstance(logits, tuple):
        logits = logits[0]

    all_logits.append(logits)
    all_labels.append(preds.label_ids)

# Concatenate all chunks
logits = np.concatenate(all_logits, axis=0)
labels = np.concatenate(all_labels, axis=0)

print("logits shape:", logits.shape)

# Convert logits to predicted token IDs
pred_ids = torch.argmax(torch.tensor(logits), dim=-1)

# Only take the first token for yes/no classification
first_token_ids = pred_ids[:, 0]

# Decode predictions and labels
decoded_preds = tokenizer.batch_decode(first_token_ids, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

print("decoded_preds sample:", decoded_preds[:20])
print("decoded_labels sample:", decoded_labels[:20])

# Map yes/no to 1/0
y_pred = [1 if x.strip().lower() == "yes" else 0 for x in decoded_preds]
y_true = [1 if x.strip().lower() == "yes" else 0 for x in decoded_labels]

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)


In [ ]:
preds = trainer.predict(test_dataset)
print(preds)
y_pred = preds.predictions.argmax(-1)
y_true = preds.label_ids

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

In [ ]:
y_preds = preds.predictions.argmax(-1)
count_predict_1 = 0
count_label_1 = 0
for i in range(len(test_dataset)):
    if i < 20:
        print(sum(test_dataset[i]["attention_mask"]), test_dataset[i]["labels"], y_preds[i])
    count_predict_1 += y_preds[i]
    count_label_1 += test_dataset[i]["labels"].item()

print(count_predict_1, "/", len(test_dataset), count_predict_1 / len(test_dataset))
print(count_label_1, "/", len(test_dataset), count_label_1 / len(test_dataset))